In [1]:
spark.stop()

NameError: name 'spark' is not defined

# Setting paths

In [1]:
import matplotlib
matplotlib.use('Agg')
import os, sys
import datetime as dt
DEVEL_SRC = os.path.join(os.environ.get('BDA_USER_HOME', ''), "src", "devel")
if DEVEL_SRC not in sys.path:
    sys.path.append(DEVEL_SRC)

USECASES_SRC = os.path.join(DEVEL_SRC, "use-cases") # TODO when - is removed, remove also this line and adapt imports
if USECASES_SRC not in sys.path: 
    sys.path.append(USECASES_SRC)
    
AMDOCS_SRC = os.path.join(DEVEL_SRC, "amdocs_informational_dataset") # TODO when - is removed, remove also this line and adapt imports
if AMDOCS_SRC not in sys.path: 
    sys.path.append(AMDOCS_SRC)
    
import pykhaos.utils.custom_logger as clogger
logging_file = os.path.join(os.environ.get('BDA_USER_HOME', ''), "logging",
                                    "out_" + dt.datetime.now().strftime("%Y%m%d_%H%M%S") + ".log")
logger = clogger.configure_logger(log_filename=logging_file, std_channel=sys.stderr, logger_name="")
logger.info("Logging to file {}".format(logging_file))    
    
EXTERNAL_PATH = os.path.join(os.environ.get('BDA_USER_HOME', ''), "src", "devel", "pykhaos", "external_lib")
if EXTERNAL_PATH not in sys.path:
    sys.path.append(EXTERNAL_PATH)

import pykhaos.utils.notebooks as nb

RUNNING_FROM_NOTEBOOK = nb.isnotebook()
import matplotlib.pyplot as plt
if RUNNING_FROM_NOTEBOOK:
    %load_ext autoreload
    %autoreload 2
    %matplotlib inline  
    
if not RUNNING_FROM_NOTEBOOK:
    args = my_project.arg_parser()

    

20190620-161140 [INFO ] Logging to file /var/SP/data/home/csanc109/logging/out_20190620_161140.log


In [2]:
import time
start_time = time.time()
app_name = "delivery_checks"

import pykhaos.utils.pyspark_configuration as pyspark_config
sc, spark, sql_context = pyspark_config.get_spark_session(app_name=app_name, log_level="OFF", min_n_executors = 1, max_n_executors = 10, n_cores = 4,
                         executor_memory = "16g", driver_memory="4g")
print("Ended spark session: {} secs | default parallelism={}".format(time.time() - start_time,
                                                                     sc.defaultParallelism))

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

Ended spark session: 37.6309781075 secs | default parallelism=2


In [3]:
from amdocs_informational_dataset.engine.call_centre_calls import CallCentreCalls
from pyspark.sql.functions import collect_set, concat, size, coalesce, col, lpad, struct, count as sql_count, lit, min as sql_min, max as sql_max, collect_list, udf, when, desc, asc, to_date, create_map, mean as sql_mean, sum as sql_sum
from pyspark.sql.types import StringType, ArrayType, MapType, StructType, StructField, IntegerType
from pyspark.sql.functions import array, regexp_extract
from itertools import chain
import pandas as pd

from pyspark.sql.functions import greatest, lower, upper, trim
from pyspark.sql.functions import collect_set, col, lpad, lit, collect_list, desc, asc, mean as sql_mean, sum as sql_sum, datediff
#from pyspark.sql.types import StringType, ArrayType, MapType, StructType, StructField, IntegerType
from pyspark.sql.functions import array, regexp_extract, datediff, to_date, from_unixtime, unix_timestamp, desc, when, col, lit, udf, size, \
    array, isnan, upper, coalesce, length, lower, concat, create_map, sum as sql_sum, greatest, max as sql_max, sort_array
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
import re
from pyspark.sql.types import ArrayType, FloatType, StringType
from pyspark.sql.functions import datediff
import numpy as np


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# METADATA CHECKS

In [ ]:
df_extra_feats = spark.read.load("/data/udf/vf_es/churn/extra_feats_mod/extra_feats/year=2019/month=3/day=21")
[col_ for col_ in df_extra_feats.columns if col_.startswith("navigation_masmovil_w2")]


In [ ]:
df_metadata = spark.read.option("delimiter", "|").option("header", True).csv("/data/udf/vf_es/churn/metadata/metadata_v1.2")
#df_metadata.select("fea").groupby("source").agg(sql_count("*")).show()

In [ ]:
df_netapps = spark.read.load("/data/udf/vf_es/churn/extra_feats_mod/additional/year=2018/month=12/day=31/")
df_netapps.columns

In [ ]:
[col_ for col_ in df_netapps.columns if "ebay_days" in col_]

In [ ]:
df_metadata.where(col("feature").rlike("netapps_ns_apps_ebay_days")).show(100,truncate=False)

In [ ]:
df_metadata.select("feature", "imputation").where(col("feature").rlike("navigation_masmovil_w2")).show(50, truncate=False)


In [ ]:
starting_day = "20181114"
closing_day = "20190114"

from pykhaos.utils.date_functions import move_date_n_cycles

ss = starting_day
while ss <= closing_day:
    print(ss)
    ss = move_date_n_cycles(ss,1)

In [ ]:
df.select([count(when(isNull(c), 1)).alias(c) for c in ["FECHA_MIGRACION"]])

In [ ]:
# cols_comerciales = [col for col in ccc_id_grouped.columns
#                 if (col.startswith('sum_CCC_Pagar_menos') or
#                     col.startswith('sum_CCC_Ofrecimiento') or
#                     col.startswith('sum_CCC_Precios') or
#                     col.startswith('sum_CCC_Factura') or
#                     col.startswith('sum_CCC_Productos')
#                    )]
# cols_servicio = [col for col in ccc_id_grouped.columns
#                 if (col.startswith('sum_CCC_Incidencia') or
#                     col.startswith('sum_CCC_Averia') or
#                     col.startswith('sum_CCC_Consulta_Tecnica') or
#                     col.startswith('sum_CCC_Provision') or
#                     col.startswith('sum_CCC_Resultado_Escalo') or
#                     col.startswith('sum_CCC_Resultado_Envio_tecnico') or
#                     col.startswith('sum_CCC_Resultado_Transferencia') or
#                     col.startswith('sum_CCC_Resultado_Reclamacion')
#                    )]

# cols_servicio_fibra = [col for col in ccc_id_grouped.columns
#                       if (col.startswith('sum_CCC_Incidencia_Provision_Fibra') or
#                           col.startswith('sum_CCC_Averia_Fibra') or
#                           col.startswith('sum_CCC_Consulta_Tecnica_Fibra') or
#                           col.startswith('sum_CCC_Provision_Fibra')
#                          )]
# cols_servicio_neba = [col for col in ccc_id_grouped.columns
#                       if (col.startswith('sum_CCC_Incidencia_Provision_Neba') or
#                           col.startswith('sum_CCC_Averia_Neba') or
#                           col.startswith('sum_CCC_Consulta_Tecnica_Neba') or
#                           col.startswith('sum_CCC_Provision_Neba')
#                          )]
# cols_servicio_dsl = [col for col in ccc_id_grouped.columns
#                     if (col.startswith('sum_CCC_Incidencia_Provision_DSL') or
#                         col.startswith('sum_CCC_Averia_DSL') or
#                         col.startswith('sum_CCC_Consulta_Tecnica_DSL') or
#                         col.startswith('sum_CCC_Provision_DSL')
#                        )]
# cols_servicio_movil = [col for col in ccc_id_grouped.columns
#                       if (col.startswith('sum_CCC_Incidencia_Provision_Movil') or
#                           col.startswith('sum_CCC_Averia_Movil') or
#                           col.startswith('sum_CCC_Consulta_Tecnica_Movil') or
#                           col.startswith('sum_CCC_Provision_Movil')
#                          )]
# cols_servicio_tv = [col for col in ccc_id_grouped.columns
#                       if (col.startswith('sum_CCC_Incidencia_Provision_TV') or
#                           col.startswith('sum_CCC_Averia_TV') or
#                           col.startswith('sum_CCC_Consulta_Tecnica_TV') or
#                           col.startswith('sum_CCC_Provision_TV')
#                          )]
# cols_servicio_modemrouter = [col for col in ccc_id_grouped.columns
#                       if (col.startswith('sum_CCC_Incidencia_Provision_Modem/Router') or
#                           col.startswith('sum_CCC_Averia_Modem/Router') or
#                           col.startswith('sum_CCC_Consulta_Tecnica_Modem/Router') or
#                           col.startswith('sum_CCC_Provision_Modem/Router')
#                          )]

In [ ]:
df_all = df_all.withColumn("is_fraud", when(col("INT_RAZON").rlike("(?i)enga.o comercial|venta enga.osa"),1).otherwise(0))
df_all = df_all.withColumn("Queja_Trato", when(col("INT_SUBTIPO").isin(["Queja trato", "Quejas sevicios de atencion"]),1).otherwise(0))

In [ ]:
df_all.where(col("CATEGORY_1")=="NA").select("INT_Tipo", "INT_Tipo").groupBy("INT_Tipo", "INT_Tipo").agg(sql_count("*").alias("count")).sort(desc("count")).show(30, False)

In [ ]:
from pyspark.sql.functions import desc
df_all.where(col("CATEGORY_1")=="NA").select("INT_Tipo", "INT_Tipo","INT_Resultado").groupBy("INT_Tipo", "INT_Tipo","INT_Resultado").agg(sql_count("*").alias("count")).sort(desc("count")).show(30, False)

In [ ]:
df_scores_incidencias.columns

In [79]:
from churn.analysis.

['num_cliente_customer',
 'cod_estado_general',
 'clase_cli_cod_clase_cliente',
 'msisdn',
 'num_cliente_service',
 'campo2',
 'rgu',
 'srv_basic']

In [ ]:

CLOSING_DAY = "20181231" 
from pykhaos.utils.date_functions import move_date_n_days, move_date_n_yearmonths
ccc_start_date_ = move_date_n_days(CLOSING_DAY, -30, str_fmt="%Y%m%d")



# # Use the previous YYYYMM to get the service problems
# yyyymm_str = move_date_n_yearmonths(CLOSING_DAY[:6], -1)
# print("Using yyyymm={} to obtain the service_problems".format(yyyymm_str))

# from churn.datapreparation.general.problemas_servicio_data_loader import get_service_problems

# df_service_problems = get_service_problems(spark, CLOSING_DAY)

In [ ]:
from churn.datapreparation.general.tgs_data_loader import get_tgs

CLOSING_DAY = "20181231"
yyyymm_str = move_date_n_yearmonths(CLOSING_DAY[:6], -1)
print("Using yyyymm={} to obtain the service_problems".format(yyyymm_str))


In [ ]:
from reportlab.lib.enums import TA_RIGHT

In [ ]:
from churn.datapreparation.app.generate_table_extra_feats import impute_nulls
df_tgs_nonulls = impute_nulls_hola(df_tgs, spark)

In [ ]:
from pykhaos.utils.pyspark_utils import count_nans

A = count_nans(df_tgs_nonulls, as_pandas=False)
B=A.rdd.collect()[0]
{col_:val for col_,val in B.asDict().items() if val>0} 

In [ ]:
mycols = [col_ for col_ in df_kk.columns if col_.startswith("ccc_num")]
df_kk.select(mycols).dtypes

In [ ]:
[col_ for col_, type_ in df_extra_feats.dtypes if type_ == "string" ]

In [ ]:
df_ccc = spark.read.load("/data/udf/vf_es/churn/extra_feats_mod/extra_feats/year=2019/month=3/day=14")



from pykhaos.utils.pyspark_utils import count_nans

A = count_nans(df_ccc, as_pandas=False)
B=A.rdd.collect()[0]
{col_:val for col_,val in B.asDict().items() if val>0} 

In [ ]:
[col_ for col_ in df_ccc.columns if "ccc_num"in col_]


In [ ]:
import re

re.match("^(.*)use-cases(.*)", "/var/SP/data/bdpmdses/deliveries_churn/version1/use-cases/churn/delivery").group(1)

In [ ]:
df = spark.sql("select msisdn, scoring, model_executed_at from parquet.`/data/attributes/vf_es/model_outputs/model_scores/model_name=churn_preds_mobileandfbb` where predict_closing_date = '20190430'")
df.show()

In [ ]:
print(checker_scores_test_obj)

# NEW CHECK SECTIONS

In [ ]:
import yaml

input_config = yaml.load(open("/var/SP/data/home/csanc109/src/devel/use-cases/churn/delivery/delivery_config.yaml"))
input_config

STEP_CAR = "car"
STEP_DELIVERY_FILE = "delivery_file"
STEP_DELIVERY_TABLE = "delivery_table"
STEP_DP_LEVERS = "dp_levers"
STEP_EXTRAFEATS = 'extrafeats'
STEP_JOIN_PREV_CAR = 'join_prev_car'
STEP_PREPARED_CAR = 'prepared_car'
STEP_SCORES = "scores"
STEP_SCORES_MOBILEANDFBB = 'scores_mobileandfbb'
STEP_SCORES_ONLYMOB = 'scores_onlymob'
STEP_SCORES_OTHERS = 'scores_others'
STEP_MODEL_EVAL = "model_evaluation"
STEP_CHURN_REASONS = "churn_reasons"


input_config

In [4]:
# closing_day = "20181231"
# tr_cycle_ini = "20181031"
# tr_cycle_end = "20181031"
# starting_day = "20181001"

closing_day = "20190607"
tr_cycle_ini = None
tr_cycle_end = None
starting_day = "20190508"
ref_date = "20190606"

# closing_day = "20190430"
# tr_cycle_ini = None
# tr_cycle_end = None
# starting_day = "20190401"

scores_segments_list="onlymob,mobileandfbb,others".split(",")

deliv_file_format = "reasons".split(",")

training_closing_date=None
if tr_cycle_ini and tr_cycle_end:
    training_closing_date = tr_cycle_ini + "to" + tr_cycle_end


from churn.others.run_churn_delivery_scores_incidences_levers_master import get_columns_required
columns_required = get_columns_required(deliv_file_format)

print(columns_required)

horizon  = 8

check = True
delivery = False

NUM_RETRIES_STEP = 2
NUM_RETRIES_PROCESS = 2

from pykhaos.utils.date_functions import move_date_n_cycles

prev_closing_day = move_date_n_cycles(closing_day, n=-1)

from churn.delivery.checkers_class import *
from churn.delivery.printers_class import * 
from functools import partial
from churn.delivery.delivery_constants import *
from churn.delivery.checkers_class import checker, check_join_car, check_extra_feats, check_scores_new, \
    check_levers_model, check_car, check_car_preparado, check_delivery_file, check_delivery_table
from churn.delivery.printers_class import *  # print_msg, format_modules_msg, SECTION_CAR, format_table_msg
from functools import partial

checkers_list = []

checker_car_obj = checker(spark, partial(check_car, spark, closing_day), partial(format_modules_msg), closing_day,
                          SECTION_CAR)
checker_join_prev_car_obj = checker(spark, partial(check_join_car, spark, prev_closing_day),
                                    partial(format_table_msg), closing_day, SECTION_JOIN_CAR)
checker_extra_feats_obj = checker(spark, partial(check_extra_feats, spark, closing_day),
                                  partial(format_modules_msg), closing_day, SECTION_EXTRA_FEATS)

checker_prepared_car_obj = checker(spark, partial(check_car_preparado, spark, closing_day),
                                   partial(format_table_msg), closing_day, SECTION_CAR_PREPARADO)

checker_scores_onlymob_obj = checker(spark, partial(check_scores_new, spark, closing_day, segment="onlymob",
                                                    tr_cycle_ini=tr_cycle_ini,
                                                    tr_cycle_end=tr_cycle_end), partial(format_table_msg),
                                     closing_day, SECTION_SCORES_NEW + " " + "onlymob")

checker_scores_mobileandfbb_obj = checker(spark,
                                          partial(check_scores_new, spark, closing_day, segment="mobileandfbb",
                                                  tr_cycle_ini=tr_cycle_ini,
                                                  tr_cycle_end=tr_cycle_end), partial(format_table_msg),
                                          closing_day, SECTION_SCORES_NEW + " " + "mobileandfbb")

checkers_list = [checker_car_obj, checker_join_prev_car_obj, checker_extra_feats_obj, checker_prepared_car_obj,
                 checker_scores_onlymob_obj,
                 checker_scores_mobileandfbb_obj
                ]

if "others" in scores_segments_list:
    checker_scores_others_obj = checker(spark,
                                              partial(check_scores_new, spark, closing_day, segment="others",
                                                      tr_cycle_ini=tr_cycle_ini,
                                                      tr_cycle_end=tr_cycle_end), partial(format_table_msg),
                                              closing_day, SECTION_SCORES_NEW + " " + "others")
    checkers_list.append(checker_scores_others_obj)
else:
    checker_scores_others_obj = None

if "palanca" in columns_required:
    checker_dp_levers_obj = checker(spark, partial(check_levers_model, spark, closing_day), partial(format_table_msg),
                                    closing_day, SECTION_DP_LEVERS)

    checkers_list.append(checker_dp_levers_obj)
else:
    checker_dp_levers_obj = None



checker_churnreasons_table_obj = checker(spark, partial(check_churn_reasons_table, spark, closing_day, scores_segments_list, training_closing_date),
                                     partial(format_table_msg),
                                     closing_day, SECTION_DP_CHURN_REASONS)
    
checkers_list.append(checker_churnreasons_table_obj)

checker_delivery_table_obj = checker(spark, partial(check_delivery_table, spark, closing_day, training_closing_date), partial(format_table_msg),
                                        closing_day, SECTION_DELIV_TABLE)

checkers_list.append(checker_delivery_table_obj)

checker_delivery_file_obj = checker(spark, partial(check_delivery_file, closing_day, tr_cycle_ini, tr_cycle_end, horizon, deliv_file_format),
                                    partial(format_table_msg), closing_day, SECTION_DELIV_FILE)

checkers_list.append(checker_delivery_file_obj)

checker_mod_out_delivery_entry_obj = checker(spark, partial(check_model_output_entry, spark, MODEL_NAME_DELIV_CHURN, closing_day, training_closing_date),
                                            partial(format_table_msg), closing_day, SECTION_CHURN_DELIV_ENTRY)

checkers_list.append(checker_mod_out_delivery_entry_obj)


checker_model_evaluation_obj = checker(spark, partial(check_model_evaluation, move_date_n_cycles(closing_day, n=-4), MODEL_EVALUATION_NB_DAYS, ref_date),
                                    partial(format_table_msg), closing_day, SECTION_MODEL_EVALUATION)

checkers_list.append(checker_model_evaluation_obj)


def run_checker(checkers_list):
    for checker_obj in checkers_list:
        checker_obj.check_func()
       
    
print(" ")
print(" ")
print(" ")
print(" ")
print(" ")
print(" ")
    
    
def print_checkers(checkers_list):
    for checker_obj in checkers_list:
        checker_obj.print_msg()
   
run_checker(checkers_list)

print(" ")
print(" ")
print(" ")
print(" ")
print(" ")
print(" ")


print_checkers(checkers_list)



print(" ")
print(" ")
print(" ")
print(" ")
print(" ")
print(" ")

20190620-161221 [INFO ] STEP CAR ) checking started...


['Incertidumbre', 'comb_score', 'top1_reason', 'msisdn', 'top2_reason', 'top4_reason', 'top3_reason', 'top0_reason', 'top5_reason', 'comb_decile', 'IND_TIPIF_UCI']
 
 
 
 
 
 


20190620-161233 [INFO ] STEP JOIN_CAR ) checking started...
20190620-161236 [INFO ] STEP EXTRA_FEATS ) checking started...
20190620-161236 [INFO ]    | Check will include following modules: ccc,tgs,pbma_srv,order,devices,delta,campaign,comp_web,netscout,additional,extra_feats
20190620-161245 [INFO ] STEP CAR PREPARADO ) checking started...
20190620-161246 [INFO ] STEP SCORES onlymob ) checking started...
20190620-161246 [INFO ] select msisdn, scoring, model_executed_at from parquet.`/data/attributes/vf_es/model_outputs/model_scores/model_name=churn_preds_onlymob` where predict_closing_date = '20190607'
20190620-161313 [INFO ] ERROR - SCORES segment=onlymob. Not found any scores for closing_day=20190607 tr_ini=None tr_end=None
20190620-161313 [INFO ] STEP SCORES mobileandfbb ) checking started...
20190620-161313 [INFO ] select msisdn, scoring, model_executed_at from parquet.`/data/attributes/vf_es/model_outputs/model_scores/model_name=churn_preds_mobileandfbb` where predict_closing_date

20190620-161540 [INFO ]   [OK] CAR PREPARADO       | INFO - CAR PREPARADO 'tests_es.jvmm_amdocs_prepared_car_mobile_complete_20190607' exists
20190620-161540 [INFO ]   {KO} SCORES onlymob      | ERROR - SCORES segment=onlymob. Not found any scores for closing_day=20190607 tr_ini=None tr_end=None
20190620-161540 [INFO ]   {KO} SCORES mobileandfbb | ERROR - SCORES segment=mobileandfbb. Not found any scores for closing_day=20190607 tr_ini=None tr_end=None
20190620-161540 [INFO ]   {KO} SCORES others       | ERROR - SCORES segment=others. Not found any scores for closing_day=20190607 tr_ini=None tr_end=None
20190620-161540 [INFO ]   {0%} CHURN REASONS       | Summary:
20190620-161540 [INFO ]                              > KO - Path /data/udf/vf_es/churn/churn_reasons/churn_reasons_results does not have any content for segment='onlymob' closing_day='20190607' training='None' exists
20190620-161540 [INFO ]                              > KO - Path /data/udf/vf_es/churn/churn_reasons/churn_rea

In [ ]:
from churn.analysis.triggers.customer_master import get_target

closing_day  = "20190430"
df_services = get_target(spark, closing_day)
df_services.show()

20190430
20190531
[Info getFixPortRequestsForMonth] Thu Jun 20 17:03:50 2019 Port-out requests for fixed services during period 20190430-20190531: 57461


# ORDERS AND SLA

In [13]:
# df_order = spark.read.load("/data/udf/vf_es/amdocs_ids/orders/year=2019/month=5/day=14")
# from churn.datapreparation.engine.ccc_model_auxiliar import prepare_orders
# df_order = prepare_orders(df_order, "20190514")


# values= ['Order_N{}'.format(n) for n in range(1,11)] # 1 to 10

# from pyspark.sql.functions import datediff

# for vv in values:
#     df_order = (df_order
#                 #.withColumn(vv + "_CompletedDate", when(datediff(lit("1900-01-01 00:00:00"), col(vv + "_CompletedDate")).cast("double") > 0, None).otherwise(vv + "_CompletedDate"))
#                 .withColumn("duration_{}".format(vv), when(   (col(vv + "_CompletedDate").isNotNull()) & (col(vv + "_StartDate").isNotNull()),
#                                                                            datediff(col(vv + "_CompletedDate"), col(vv + "_StartDate"))).otherwise(-1))
#            )
    

In [61]:
closing_day  = "20190514"
from pykhaos.utils.date_functions import move_date_n_days
starting_day = move_date_n_days(closing_day, n=-365)

In [62]:
data_order_ori = (spark.read.load("/data/raw/vf_es/customerprofilecar/ORDERCRMOW/1.1/parquet/")                   
                   .where(col('fecha_entrada') <= closing_day)
                   .where(col('fecha_entrada') >= starting_day)
                  )

w_orderclass = Window().partitionBy("OBJID").orderBy(desc("year"), desc("month"), desc("day"))

data_orderclass_ori = (spark.read.load("/data/raw/vf_es/customerprofilecar/ORDERCLASOW/1.0/parquet/")
                       .where(concat(col('year'), lpad(col('month'), 2, '0'), lpad(col('day'), 2, '0')) <= closing_day)
                       .withColumn("rowNum", row_number().over(w_orderclass))
                       .where(col('rowNum') == 1)
                       .drop(*['year', 'month', 'day'])
                       )

w_order = Window().partitionBy("orden_id").orderBy(desc("year"), desc("month"), desc("day"))

data_orders_joined = (
                        data_order_ori.join(data_orderclass_ori, on=col('clase_orden') == col('objid'), how='leftouter')
                                      .withColumn("rowNumorder", row_number().over(w_order))
                                      .where(col('rowNumorder') == 1)
             )


import re
def convert(s):
    if not s: return ""
    s = re.sub("instalaci.n", "instalacion", s)
    s = re.sub("m.vil", "movil", s)
    s = re.sub("desconexi.n", "desconexion", s)
    s = re.sub("disminuci.n", "disminucion", s)
    return s

convert_udf = udf(convert,StringType())

data_orders_joined = data_orders_joined.withColumn('X_TIPO_ORDEN', trim(upper(col('X_TIPO_ORDEN'))))
data_orders_joined = data_orders_joined.withColumn('x_clasificacion', trim(lower(col('x_clasificacion'))))
data_orders_joined = data_orders_joined.withColumn('X_Description', trim(lower(col('X_Description'))))
data_orders_joined = data_orders_joined.withColumn("X_Description", convert_udf(col("X_Description")))
data_orders_joined = (data_orders_joined.withColumn("days_since_start", when(col("FECHA_ENTRADA").isNotNull(),
                                                                        datediff(from_unixtime(unix_timestamp(lit(closing_day), "yyyyMMdd")), col("FECHA_ENTRADA"))).otherwise(-1)))

data_orders_joined = (data_orders_joined.withColumn("days_since_completed", when(col("FECHA_WO_COMPLETA").isNotNull() & (col("ESTADO_FIN_WO")=="C"),#be sure is completed
                                                                        datediff(from_unixtime(unix_timestamp(lit(closing_day),
                                                                                           "yyyyMMdd")), col("FECHA_WO_COMPLETA"))).otherwise(None))
                                        # set to None, orders completed after closing_day
                                        .withColumn("days_since_completed", when(col("days_since_completed")<0, None).otherwise(col("days_since_completed")))                     
                     )

# -----[----------------]------
#     a year        closing_day
data_orders_joined = (data_orders_joined.where( (col("days_since_start")<=365) & (col("days_since_start")>=0)))

data_orders_joined = (data_orders_joined
                          .withColumn("duration",
                                when((col("days_since_start")!=-1) & (col("days_since_completed").isNotNull()),
                                     (col("days_since_start") - col("days_since_completed"))).otherwise(None))
                     )


In [73]:
# remove orders with category "desconexion" and "reconexion"
data_orders_joined = data_orders_joined.where(~col("x_clasificacion").isin(["desconexion", "reconexion"]))


# within the category "disminucion", remove those that contain "portabilidad" y "baja" inside the X_Description field
data_orders_joined = data_orders_joined.where(  ~((col("x_clasificacion").isin(["disminucion"])) &  
                             (col("X_Description").rlike("(?i)portabilidad|baja"))))


In [74]:
data_orders_joined.select('X_TIPO_ORDEN', 'x_clasificacion', 'X_Description').groupby('X_TIPO_ORDEN', 'x_clasificacion', 'X_Description').agg(sql_count("*").alias("count")).sort(desc("count")).show(100)

+------------+--------------------+--------------------+-------+
|X_TIPO_ORDEN|     x_clasificacion|       X_Description|  count|
+------------+--------------------+--------------------+-------+
|         COF|ord. administrativas|corrección en ofi...|1017466|
|         MVA|             aumento|modificacion de s...| 925939|
|         PTP|ord. administrativas|facturacion canal...| 661989|
|         CPP|              cambio|cambio de plan de...| 559115|
|         APP|         instalacion|instalacion movil...| 495446|
|         SIM|              cambio|suspension por im...| 487709|
|         PRO|ord. administrativas|activacion promocion| 471772|
|         FTM|ord. administrativas|    factura terminal| 417661|
|         RIM|              cambio|reconexion tras s...| 391892|
|         MVA|             aumento|     provision volte| 348785|
|         MVA|         disminucion|modificacion de s...| 319783|
|         FID|ord. administrativas|fidelización cliente| 318041|
|         PRC|         in

In [ ]:
#data_orders_joined_simple = data_orders_joined.where(col("num_cliente").isin(["478215630", "246263112", "438636469", "902332450", "080495699", "602719127", "585025928", "023917822"]))

In [ ]:
#data_orders_joined.select("num_cliente", "clase_orden", "fecha_entrada", "FECHA_WO_COMPLETA", "ESTADO_FIN_WO", "days_since_start", "days_since_completed", "duration").show(1000, truncate=False)

In [25]:
#df_order_master.where(col("duration")>col("SLA_master")).select("num_cliente", "clase_orden", "fecha_entrada", "FECHA_WO_COMPLETA", "ESTADO_FIN_WO", "days_since_start", "days_since_completed", "duration", "SLA_master").show(1000, truncate=False)

+-----------+-----------+-------------------+-------------------+-------------+----------------+--------------------+--------+----------+
|num_cliente|clase_orden|fecha_entrada      |FECHA_WO_COMPLETA  |ESTADO_FIN_WO|days_since_start|days_since_completed|duration|SLA_master|
+-----------+-----------+-------------------+-------------------+-------------+----------------+--------------------+--------+----------+
|273635434  |271462575  |2019-01-01 00:00:00|2019-01-08 00:00:00|C            |133             |126                 |7       |4         |
|755915160  |271797929  |2019-01-01 00:00:00|2019-01-08 00:00:00|C            |133             |126                 |7       |1         |
|366552281  |271003692  |2019-01-01 00:00:00|2019-01-08 00:00:00|C            |133             |126                 |7       |4         |
|074469208  |270966873  |2019-01-01 00:00:00|2019-01-11 00:00:00|C            |133             |123                 |10      |7         |
|006520553  |271462575  |2019-01-0

In [39]:
from pyspark.sql.functions import greatest, lower, upper, trim
df_sla = spark.read.option("header", True).option("delimiter", ";").option("encoding", "utf-8").csv("/data/udf/vf_es/churn/SLA/TorreControl_SLA.csv").where(col("TITULO").isNotNull()&col("CLASIFICACION").isNotNull()&col("TIPO ORDEN").isNotNull())
df_sla = df_sla.withColumn('TITULO', trim(lower(col('TITULO'))))
df_sla = df_sla.withColumn('CLASIFICACION', trim(lower(col('CLASIFICACION'))))
df_sla = df_sla.withColumn('TIPO ORDEN', trim(upper(col('TIPO ORDEN'))))

df_sla = df_sla.withColumn("TITULO", convert_udf(col("TITULO")))
df_sla = df_sla.withColumn("SLA Negocio presencial", when(col("SLA Negocio presencial").isNull(),-1).otherwise(col("SLA Negocio presencial")))
df_sla = df_sla.withColumn("SLA Negocio no presencial", when(col("SLA Negocio no presencial").isNull(),-1).otherwise(col("SLA Negocio no presencial")))
                          
df_sla = df_sla.withColumn("SLA_master",  when( col("SLA Negocio presencial").isNotNull() & col("SLA Negocio no presencial").isNotNull(), greatest(col("SLA Negocio presencial"), col("SLA Negocio no presencial")))
                                         .when( col("SLA Negocio presencial").isNotNull(), col("SLA Negocio presencial"))
                                         .when( col("SLA Negocio no presencial").isNotNull(), col("SLA Negocio no presencial")).otherwise(-1))

df_order_master = data_orders_joined.join(df_sla.select("CLASIFICACION",'TIPO ORDEN',"TITULO", "SLA_master"), 
                          on= (df_sla['TIPO ORDEN'] == data_orders_joined['X_TIPO_ORDEN']) & 
                              (df_sla["CLASIFICACION"] == data_orders_joined["x_clasificacion"]) & 
                              (df_sla["TITULO"] == data_orders_joined['X_Description']), how="left")

#df_order2.select("NUM_CLIENTE", "orden_id", "CLASIFICACION", 'TIPO ORDEN', "TITULO", "SLA_master").show(truncate=False)


In [40]:
df_order_master = df_order_master.cache()



In [41]:
#df_order_master.select("num_cliente", "clase_orden", "fecha_entrada", "FECHA_WO_COMPLETA", "ESTADO_FIN_WO", "days_since_start", "days_since_completed", "duration", "SLA_master").sort(asc("days_since_start")).show(100)

In [42]:
df_order = df_order_master

In [28]:
# num_cliente con ordenes fuera de sla
df_order = df_order_master.where(col("num_cliente").isin(["100627722", "100627722", "101512463", "103290455","105714689","106869490","106954619","107024721","107995794",
                                                          "107995794","108892672",
                                                          "109687054","109823680",
                                                          "111003926","111003926","113626259","114392826","114680682","115194594","116940756","117184992","117256337",
                                                          "118624850","119569754",
                                                          "120395699","124026200",
                                                          "126634771","127762809","127953977",
                                                          "129389505","130888049","131223736","131589961","132312669","135375447","135708854","135708854","136198452","136246120",
                                                          "136339118","136610373","136610373","139975390"]))

In [14]:
# 'FECHA_ENTRADA',
#  'FECHA_VENTA',
#  'FECHA_WO_COMPLETA'


In [45]:
from pyspark.sql.types import FloatType
# keep_gt_factor_udf = udf(lambda milista, factor: list([milista[ii] for ii in range(len(milista)) if milista[ii] > factor]), ArrayType(StringType()))
days_range = [30, 60, 90, 120, 180, 240, 365]
# remove_undefined_udf = udf(lambda milista: list([milista[ii] for ii in range(len(milista)) if milista[ii] != -1]), ArrayType(FloatType()))
# avg_days_bw_orders_udf = udf(lambda milista: float(np.mean([milista[ii + 1] - milista[ii] for ii in range(len(milista) - 1)])), FloatType())

# mean_udf = udf(lambda milista: float(np.mean(milista)), FloatType())
from pyspark.sql.functions import lag, mean as sql_avg


import numpy as np

for dd in days_range:
    
        df_order = df_order.withColumn("duration_last{}".format(dd), when( (col("days_since_completed")<=dd) & (col("days_since_completed")!=-1), col("duration")).otherwise(None))
        df_order = df_order.withColumn("SLA_factor_last{}".format(dd), when(col("duration_last{}".format(dd)).isNotNull() & col("SLA_master").isNotNull(),
                                                                            col("duration_last{}".format(dd))/col("SLA_master")).otherwise(None))
        # to ignore an order, set the column "days_since..." to -1. following line, set "diff" value of days_since_started=-1 to Null
        # diff is the difference (in days) between an order and the next one
        df_order = df_order.withColumn("diff", when(col("days_since_start")!=-1, lag(col("days_since_start"),-1).over(Window.partitionBy("num_cliente").orderBy(asc("days_since_start")))-col("days_since_start")).otherwise(None))
                                       
                                       
                                       

In [ ]:
df_order_master.select("num_cliente", "clase_orden", "fecha_entrada", "FECHA_WO_COMPLETA", "ESTADO_FIN_WO", "days_since_start", "days_since_completed", "duration", "SLA_master").sort(asc("days_since_start")).show(100)

In [33]:

ccols = ["num_cliente", "clase_orden", "fecha_entrada", "FECHA_WO_COMPLETA", "ESTADO_FIN_WO", "days_since_start", "days_since_completed"] + [col_ for col_ in df_order.columns if col_.startswith("duration") or col_.startswith("SLA")]

df_order.select(ccols).show(500)

+-----------+-----------+-------------------+-------------------+-------------+----------------+--------------------+----+--------+----------+---------------+-----------------+---------------+-----------------+---------------+------------------+----------------+-------------------+----------------+-------------------+----------------+-------------------+----------------+-------------------+
|num_cliente|clase_orden|      fecha_entrada|  FECHA_WO_COMPLETA|ESTADO_FIN_WO|days_since_start|days_since_completed| SLA|duration|SLA_master|duration_last30|SLA_factor_last30|duration_last60|SLA_factor_last60|duration_last90| SLA_factor_last90|duration_last120| SLA_factor_last120|duration_last180| SLA_factor_last180|duration_last240| SLA_factor_last240|duration_last365| SLA_factor_last365|
+-----------+-----------+-------------------+-------------------+-------------+----------------+--------------------+----+--------+----------+---------------+-----------------+---------------+-----------------+--

In [48]:
df_agg = (df_order.groupby("num_cliente").agg( *([sql_count(when( ((col("days_since_start")<dd) & (col("days_since_start")!=-1)), col("days_since_start")).otherwise(None)).alias("nb_started_orders_last{}".format(dd)) for dd in days_range] + 
                                                 [sql_count(when(col("duration_last{}".format(dd)).isNotNull(), col("duration_last{}".format(dd))).otherwise(None)).alias("nb_completed_orders_last{}".format(dd)) for dd in days_range] + 
                                                 [sql_count(when(col("SLA_factor_last{}".format(dd))>ff, col("SLA_factor_last{}".format(dd))).otherwise(None)).alias("nb_completed_orders_last{}_{}SLA".format(dd, ff)) for dd in days_range for ff in [1,2,3]] +
                                                     # [sql_count(when(col("SLA_factor_last{}".format(dd))>1, col("SLA_factor_last{}".format(dd))).otherwise(None)).alias("nb_completed_orders_last{}_{}SLA".format(dd, 1)) for dd in days_range] 
                                                 # diff=null entries are ignored for average
                                                 [sql_avg(col("diff")).alias("avg_days_bw_open_orders")] + 
                                                 # ignore null factors (when SLA_master is null)
                                                 [sql_avg(col("SLA_factor_last{}".format(dd))).alias("mean_sla_factor_last{}".format(dd)) for dd in days_range] 
                                             )))
        


In [36]:
df_agg.show(100)

+-----------+------------------------+------------------------+------------------------+-------------------------+-------------------------+-------------------------+-------------------------+--------------------------+--------------------------+--------------------------+---------------------------+---------------------------+---------------------------+---------------------------+-------------------------------+-------------------------------+-------------------------------+-------------------------------+-------------------------------+-------------------------------+-------------------------------+-------------------------------+-------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+--------------------------------+------------------------------

In [127]:
# for ii in range(1,11):
#     df_order = (df_order.join(df_order_master.select("ORDEN_ID","SLA_master"), 
#                  on=(df_order["Order_N{}_Id".format(ii)]==df_order_master["ORDEN_ID"]), how="left"))
#     df_order = df_order.drop("ORDER_ID") 
    
#     df_order = df_order.withColumnRenamed("SLA_master", "Order_N{}_SLA".format(ii))

In [16]:
df_order_car = spark.read.load("/data/udf/vf_es/amdocs_inf_dataset/customer_agg/year=2019/month=5/day=14")
df_order_car.show()

+-----------+------------------------+------------------------+---------------------------+---------------------------+-----------------------+-----------------------+----------------------------+----------------------------+-------------------------------+-------------------------------+--------------------------+--------------------------+------------------------+------------------------+------------------------+-----------------------+-------------------------+-------------------------+----------------------------+----------------------------+------------------------+------------------------+-----------------------------+-----------------------------+--------------------------------+--------------------------------+---------------------------+---------------------------+-------------------------+-------------------------+-------------------------+------------------------+-----------------------------------+-----------------------------+------------------------+----------------------

In [ ]:

df_agg = df_agg.withColumn("day", lit(int(closing_day[6:])))
df_agg = df_agg.withColumn("month", lit(int(closing_day[4:6])))
df_agg = df_agg.withColumn("year", lit(int(closing_day[:4])))

path_to_save = "/data/attributes/vf_es/trigger_analysis/orders_sla/"

(df_agg
    .write
    .partitionBy('year', 'month', 'day')
    .mode("append")
    .format("parquet")
    .save(to_save))


In [126]:
from pyspark.sql.types import FloatType
keep_gt_factor_udf = udf(lambda milista, factor: list([milista[ii] for ii in range(len(milista)) if milista[ii] > factor]), ArrayType(StringType()))
days_range = [30, 90, 120, 180, 240, 360]
remove_undefined_udf = udf(lambda milista: list([milista[ii] for ii in range(len(milista)) if milista[ii] != -1]), ArrayType(FloatType()))
avg_days_bw_orders_udf = udf(lambda milista: float(np.mean([milista[ii + 1] - milista[ii] for ii in range(len(milista) - 1)])), FloatType())

mean_udf = udf(lambda milista: float(np.mean(milista)), FloatType())

values= ['Order_N{}'.format(n) for n in range(1,11)] # 1 to 10

import numpy as np

for dd in days_range:

    for vv in values:
    
        df_order = df_order.withColumn("duration_last{}_{}".format(dd, vv), when( (col("days_since_{}_CompletedDate".format(vv))<=dd) & (col("days_since_{}_CompletedDate".format(vv))>0), 
                                                                                          col("duration_{}".format(vv))).otherwise(-1))

        df_order = df_order.withColumn("SLA_factor_last{}_{}".format(dd, vv), when(col("duration_last{}_{}".format(dd, vv)) != -1, col("duration_last{}_{}".format(dd, vv))/col("{}_SLA".format(vv))).otherwise(-1))

        df_order = (df_order.withColumn("aux_array", array([_col for _col in df_order.columns if _col.startswith("duration_last{}".format(dd))]))
                    .withColumn("aux_array", remove_undefined_udf(col("aux_array")))
                    .withColumn("nb_started_orders_last{}".format(dd), when(col("aux_array").isNotNull(), size(col("aux_array"))).otherwise(0))
                    .withColumn("avg_days_bw_orders_last{}".format(dd), when(col("nb_started_orders_last{}".format(dd)) > 1, avg_days_bw_orders_udf(col("aux_array"))).otherwise(-1.0))
                    .withColumn("aux_array", array([_col for _col in df_order.columns if _col.startswith("SLA_factor_last{}".format(dd))]))
                    .withColumn("aux_array", remove_undefined_udf(col("aux_array")))
                    .withColumn("mean_factor_last{}".format(dd), mean_udf(col("aux_array"))))

        for ff in [1,3]:
            df_order = (df_order
                      .withColumn("nb_completed_orders_last{}_{}SLA".format(dd, ff), when(col("nb_started_orders_last{}".format(dd)) > 0, size(keep_gt_factor_udf(col("aux_array"), lit(ff)))).otherwise(-1.0)))

        df_order = (df_order.drop("aux_array"))
              

In [93]:
df_order.select(*['Order_N1_Id',
 'Order_N1_Class',
 'Order_N1_Description',
 'Order_N1_Status',
 'ORDEN_ID',
 'CLASIFICACION',
 'TIPO ORDEN',
 'TITULO']).show(truncate=False)

+---------------+-------------------+--------------------------------------------------------------------------------+---------------+---------------+------------------+----------+-------------------------------------------------------+
|Order_N1_Id    |Order_N1_Class     |Order_N1_Description                                                            |Order_N1_Status|ORDEN_ID       |CLASIFICACION     |TIPO ORDEN|TITULO                                                 |
+---------------+-------------------+--------------------------------------------------------------------------------+---------------+---------------+------------------+----------+-------------------------------------------------------+
|000000150554344|Ord_Administrativas|Orden Automatica Repercusión Coste                                              |COM            |000000150554344|null              |null      |null                                                   |
|-00000002793550|Instalacion        |Migración de Si

In [89]:
df_order.where(col("duration_Order_N1")>0).where(col("days_since_Order_N1_CompletedDate")<30).select(*['Order_N1_Id',
 'duration_Order_N1',
 'days_since_Order_N1_StartDate',
 'days_since_Order_N1_CompletedDate',
#  'Order_N1_Class',
#  'Order_N1_Description',
#  'Order_N1_Status',
#  'ORDEN_ID',
#  'CLASIFICACION',
#  'TIPO ORDEN',
#  'TITULO',
 'Order_N1_SLA',
 'duration_last30_Order_N1',
 'SLA_factor_last30_Order_N1',
 'nb_started_orders_last30',
 'avg_days_bw_orders_last30',
 'nb_completed_orders_last30_1SLA',
 'nb_completed_orders_last30_3SLA']).show(10,truncate=False)

+---------------+-----------------+-----------------------------+---------------------------------+------------+------------------------+--------------------------+------------------------+-------------------------+-------------------------------+-------------------------------+
|Order_N1_Id    |duration_Order_N1|days_since_Order_N1_StartDate|days_since_Order_N1_CompletedDate|Order_N1_SLA|duration_last30_Order_N1|SLA_factor_last30_Order_N1|nb_started_orders_last30|avg_days_bw_orders_last30|nb_completed_orders_last30_1SLA|nb_completed_orders_last30_3SLA|
+---------------+-----------------+-----------------------------+---------------------------------+------------+------------------------+--------------------------+------------------------+-------------------------+-------------------------------+-------------------------------+
|000000176895628|194              |212                          |18                               |1           |194                     |194.0                  

In [16]:


#data_orders_joined.select("clase_orden", "objid", "orden_id", "X_TIPO_ORDEN", "SLA").show() # orden_id equivale a 'Order_Nx_Id'


In [31]:
data_orders_joined.select("X_CLASIFICACION", # --> data_orderclass_ori
                          "TIPO_WO", # --> data_order_ori
                          'DES_ORDEN', # --> data_order_ori
                          'X_Description',# --> data_orderclass_ori
                          'CLASE_ORDEN', # --> data_order_ori
                          'X_TIPO_ORDEN'
                         ).where((col("X_TIPO_ORDEN")=="AME") | (col("TIPO_WO")=="AME")).show(truncate=False)


+---------------+-------+--------------------------------------------------------------------------------+------------------+---------------+------------+
|X_CLASIFICACION|TIPO_WO|DES_ORDEN                                                                       |X_Description     |CLASE_ORDEN    |X_TIPO_ORDEN|
+---------------+-------+--------------------------------------------------------------------------------+------------------+---------------+------------+
|instalacion    |MIG    |Migración de Sistemas Vodafone a AMDOCS                                         |alta movil prepago|000000268614327|AME         |
|instalacion    |MIG    |Migración de Sistemas Vodafone a AMDOCS                                         |alta movil prepago|000000268614327|AME         |
|instalacion    |MIG    |Migración de Sistemas Vodafone a AMDOCS                                         |alta movil prepago|000000268614327|AME         |
|instalacion    |MIG    |Migración de Sistemas Vodafone a AMDOCS      

In [38]:
# ['TIPO_WO', 'COD_TIPO_VEND', 'TIPO_SERVICIO', 'TIPO_TARIFA', 'TIPO_EQUIPO'] --> data_order_ori
# ['X_TIPO_CLIENTE', 'X_TIPO_ORDEN', 'X_SUBTIPO_CLIENTE', 'X_TIPO_CIERRE_ORDEN', 'X_TIPO_CIERRE_ORDEN_CANC'] --> data_orderclass_ori
# data_orders_joined.select(['TIPO_WO', 'X_TIPO_ORDEN']).drop_duplicates().show(100)


+-------+------------+
|TIPO_WO|X_TIPO_ORDEN|
+-------+------------+
|    SPG|         SPG|
|    WBR|         WBR|
|    WCM|         WCM|
|    CNT|         CNT|
|    WAF|         WAF|
|    RNA|         RNA|
|    WLN|         WLN|
|    XPU|         XPU|
|    NAR|         NAR|
|    CTM|         CTM|
|    FAC|         FAC|
|    SAT|         IPT|
|    VBP|         VBP|
|    PML|         PML|
|    XDV|         XDV|
|    BLL|         BLL|
|    CMU|         CAS|
|    APP|         SAP|
|       |         COS|
|       |         XPP|
|    CTS|         CTS|
|    UTT|         UTT|
|    RLI|         RLI|
|    DSV|         VBA|
|    RFP|         RFP|
|       |         AHT|
|    APX|         APX|
|    INF|         IVF|
|    ZDS|         ZDS|
|    YTI|         YTI|
|    MMP|         MTA|
|    NBK|         NBK|
|    DIS|         DIS|
|    OPL|         OPL|
|    GSP|         GSP|
|    XTI|         XTI|
|    SPS|         APS|
|    DRV|         DRV|
|    CVR|         CVR|
|    RMN|         APP|
|    TXV|  

# CHECK JOIN RESULT

In [43]:
df_order2.select("NUM_CLIENTE", "orden_id", "CLASIFICACION", 'TIPO ORDEN', "TITULO", "SLA_master").show(truncate=False)


+-----------+---------------+-------------+----------+----------------------------------------+----------+
|NUM_CLIENTE|orden_id       |CLASIFICACION|TIPO ORDEN|TITULO                                  |SLA_master|
+-----------+---------------+-------------+----------+----------------------------------------+----------+
|559462218  |-00000000107515|instalacion  |INA       |instalacion combo tfno + catv + internet|5         |
|382245710  |-00000000107666|instalacion  |INA       |instalacion combo tfno + catv + internet|5         |
|965831231  |-00000000107700|instalacion  |INA       |instalacion combo tfno + catv + internet|5         |
|344261225  |-00000000107721|instalacion  |INA       |instalacion combo tfno + catv + internet|5         |
|059029154  |-00000000108144|instalacion  |INA       |instalacion combo tfno + catv + internet|5         |
|744321541  |-00000000108372|instalacion  |INA       |instalacion combo tfno + catv + internet|5         |
|757407177  |-00000000108709|instalac

In [119]:
df_order2.select("NUM_CLIENTE", "CLASIFICACION", "x_clasificacion", 'TIPO ORDEN', "X_TIPO_ORDEN", "TITULO", "X_Description", "SLA_master").show(truncate=False)


+-----------+-------------+---------------+----------+------------+----------------------------------------+----------------------------------------+----------+
|NUM_CLIENTE|CLASIFICACION|x_clasificacion|TIPO ORDEN|X_TIPO_ORDEN|TITULO                                  |X_Description                           |SLA_master|
+-----------+-------------+---------------+----------+------------+----------------------------------------+----------------------------------------+----------+
|559462218  |instalacion  |instalacion    |INA       |INA         |instalacion combo tfno + catv + internet|instalacion combo tfno + catv + internet|5         |
|382245710  |instalacion  |instalacion    |INA       |INA         |instalacion combo tfno + catv + internet|instalacion combo tfno + catv + internet|5         |
|965831231  |instalacion  |instalacion    |INA       |INA         |instalacion combo tfno + catv + internet|instalacion combo tfno + catv + internet|5         |
|344261225  |instalacion  |instala

In [36]:
#df_order2.where(col("SLA_master").isNotNull()).count()
# 33848320
#df_order2.where(col("SLA_master").isNull()).count()
#8861634
#df_order2.where(col("SLA_master").isNotNull()).select("X_TIPO_ORDEN", "x_clasificacion", "X_Description").distinct().count()
#2075
# df_order2.where(col("SLA_master").isNull()).select("X_TIPO_ORDEN", "x_clasificacion", "X_Description").distinct().count()
#2008

33848320

In [41]:
df_order2.where(col("SLA_master").isNull()).select("X_TIPO_ORDEN", "x_clasificacion", "X_Description").distinct().show(truncate=False)

+------------+---------------+----------------------------------------------------------------+
|X_TIPO_ORDEN|x_clasificacion|X_Description                                                   |
+------------+---------------+----------------------------------------------------------------+
|WNF         |instalacion    |alta movil esencial + tfno + internet ftth                      |
|WCB         |aumento        |cambio canales deco boxtv npvr ftth                             |
|IMC         |aumento        |modificacion servicio mantenimiento cablemodem ftth             |
|OCF         |cambio         |aumento tv neba fibra(sw) + cambio de plan de precios vf        |
|ATD         |migracion      |aumento tfno + internet + cambio canales dtv + tivo             |
|WET         |desconexion    |desconexion movil esencial + tfno + cm + deco tv npvr           |
|CUD         |aumento        |aumento combo tfno + dtv + tivo + internet                      |
|MMU         |migracion      |cambio cm 

In [10]:
df_order2.where(col("SLA_master").isNull()).select("orden_id", "NUM_CLIENTE", "X_TIPO_ORDEN", "x_clasificacion", "X_Description", 'TIPO ORDEN',  "TITULO", "CLASIFICACION", "SLA_master").show(truncate=False)


+---------------+-----------+-------------+---------------+----------+------------+------+------------------+----------+
|orden_id       |NUM_CLIENTE|CLASIFICACION|x_clasificacion|TIPO ORDEN|X_TIPO_ORDEN|TITULO|X_Description     |SLA_master|
+---------------+-----------+-------------+---------------+----------+------------+------+------------------+----------+
|-00000000146380|377323280  |null         |instalacion    |null      |AME         |null  |alta movil prepago|null      |
|-00000000146451|768992110  |null         |instalacion    |null      |AME         |null  |alta movil prepago|null      |
|-00000000146609|693680431  |null         |instalacion    |null      |AME         |null  |alta movil prepago|null      |
|-00000000146948|715232819  |null         |instalacion    |null      |AME         |null  |alta movil prepago|null      |
|-00000000147320|842878454  |null         |instalacion    |null      |AME         |null  |alta movil prepago|null      |
|-00000000147331|156988744  |nul

In [33]:
# -00000001539939
# -00000000146380
order_id = "-00000000146380"
df_order2.where(col("orden_id")==order_id).select("NUM_CLIENTE", 
                                                  "X_CLASIFICACION", # --> data_orderclass_ori
                                                  "TIPO_WO", # --> data_order_ori
                                                  'DES_ORDEN', # --> data_order_ori
                                                  'X_Description',# --> data_orderclass_ori
                                                  'CLASE_ORDEN', # --> data_order_ori
                                                  'X_TIPO_ORDEN',
                        
                                                  
                                                  "CLASIFICACION", 
                                                  'TIPO ORDEN', "TITULO",
                                                  "SLA_master").show(truncate=False)


+-----------+---------------+-------+--------------------------------------------------------------------------------+------------------+---------------+------------+-------------+----------+------+----------+
|NUM_CLIENTE|X_CLASIFICACION|TIPO_WO|DES_ORDEN                                                                       |X_Description     |CLASE_ORDEN    |X_TIPO_ORDEN|CLASIFICACION|TIPO ORDEN|TITULO|SLA_master|
+-----------+---------------+-------+--------------------------------------------------------------------------------+------------------+---------------+------------+-------------+----------+------+----------+
|377323280  |instalacion    |MIG    |Migración de Sistemas Vodafone a AMDOCS                                         |alta movil prepago|000000268614327|AME         |null         |null      |null  |null      |
+-----------+---------------+-------+--------------------------------------------------------------------------------+------------------+---------------+-------

In [30]:
data_orders_joined.where(col("orden_id")==order_id).select("NUM_CLIENTE",  "x_clasificacion",  "X_TIPO_ORDEN",  "X_Description").rdd.collect()#.show(truncate=False)

[Row(NUM_CLIENTE=u'377323280', x_clasificacion=u'instalacion', X_TIPO_ORDEN=u'AME', X_Description=u'alta movil prepago')]

In [27]:
df_sla.where(col("TITULO").rlike("(?i)prepago")).select("TIPO ORDEN", "CLASIFICACION", "TITULO").rdd.collect()   #.show(truncate=False)

[Row(TIPO ORDEN=u'AXN', CLASIFICACION=u'aumento', TITULO=u'aumento movil prepago con portabilidad'),
 Row(TIPO ORDEN=u'PPT', CLASIFICACION=u'aumento', TITULO=u'aumento movil prepago con terminal con portabilidad'),
 Row(TIPO ORDEN=u'PTS', CLASIFICACION=u'aumento', TITULO=u'aumento movil prepago con terminal sin portabilidad'),
 Row(TIPO ORDEN=u'PSP', CLASIFICACION=u'aumento', TITULO=u'aumento movil prepago sin terminal con portabilidad'),
 Row(TIPO ORDEN=u'PSS', CLASIFICACION=u'aumento', TITULO=u'aumento movil prepago sin terminal sin portabilidad'),
 Row(TIPO ORDEN=u'PTS', CLASIFICACION=u'aumento', TITULO=u'aumento retencion movil prepago con terminal sin portabilidad'),
 Row(TIPO ORDEN=u'PSP', CLASIFICACION=u'aumento', TITULO=u'aumento retencion movil prepago sin terminal con portabilidad'),
 Row(TIPO ORDEN=u'DPN', CLASIFICACION=u'desconexion', TITULO=u'baja movil prepago'),
 Row(TIPO ORDEN=u'BPP', CLASIFICACION=u'disminucion', TITULO=u'baja prepago migra pre-post'),
 Row(TIPO ORDEN=

In [35]:
df_sla.where(col("TIPO ORDEN").rlike("(?i)ame")).select("TIPO ORDEN", "CLASIFICACION", "TITULO").rdd.collect()   #.show(truncate=False)

[Row(TIPO ORDEN=u'AME', CLASIFICACION=u'aumento', TITULO=u'alta movil convergente')]

In [7]:
import churn.datapreparation.general.sla_data_loader

In [13]:
from churn.datapreparation.general.sla_data_loader import get_orders_df
df_orders = get_orders_df(spark, closing_day="20190514")
df_orders.show()

KeyboardInterrupt: 

# DELIVERY TIMES

In [136]:
closing_day = "20190531"


from churn.datapreparation.general.model_outputs_manager import get_complete_scores_closingday
df = get_complete_scores_closingday(spark, closing_day)
df.withColumn("step_father", col("step_name")).replace(to_replace=step_names_dict, subset=['step_name']).show(truncate=False)

AnalysisException: u"cannot resolve '`step_name`' given input columns: [metrics_test, day, nif, scores_extra_info_headers, metrics_train, varimp, executed_at, day, extra_info, predict_closing_date, extra_info, scoring, month, time, model_output, model_path, client_id, month, time, algorithm, year, training_closing_date, author_login, prediction, year, model_executed_at, target, model_level, msisdn, metrics_path];;\n'Project [executed_at#671270, model_executed_at#671271, predict_closing_date#671272, msisdn#671273, client_id#671274, nif#671275, model_output#671276, scoring#671277, prediction#671278, extra_info#671279, time#671280, year#671281, month#671282, day#671283, model_level#671299, training_closing_date#671300, target#671301, model_path#671302, metrics_path#671303, metrics_train#671304, metrics_test#671305, varimp#671306, algorithm#671307, author_login#671308, ... 7 more fields]\n+- Filter (predict_closing_date#671272 = 20190531)\n   +- Union\n      :- Project [executed_at#671270,

In [140]:

closing_day = "20190531"
prev_closing_day = move_date_n_cycles(closing_day, n=-1)
prev_prev_closing_day = move_date_n_cycles(closing_day, n=-2)

step_names_dict = {"delta" : "extrafeats", 
                   "customer" : "car",
                   "comp_web" : "extrafeats",
                   "tnps" : "car",
                   "service" : "car",
                   "device_catalogue" : "car",
                   "order" : "extrafeats",
                   "billing": "car",
                   "orders" : "car",
                   "customer_agg": "car",
                   "additional" : "extrafeats",
                   "call_centre_calls": "car",
                   "tgs"  : "extrafeats",
                   "netscout" : "extrafeats",
                   "penalties_customer": "car",
                   "usage_geneva_roam_data": "car",
                   "usage_geneva_roam_voice": "car",
                   "usage_geneva_data" : "car",
                   "usage_geneva_voice": "car",
                   "campaigns_service": "car",
                   "campaigns_customer": "car",
                   "devices" : "extrafeats",
                   "penalties_service": "car",
                   "orders_agg": "car",
                   "perms_and_prefs": "car",
                   "pbma_srv" : "extrafeats",
                   "scores" : "extrafeats",
                   "campaign" : "extrafeats",
                   "ccc" : "extrafeats"
}

step_order_dict = {"car" :  "1", "join_prev": "2", "extrafeats": "3", "generateCar": "4", "storePreparedFeatures": "5", "getModelPredictions_onlymob" : "6.1", "getModelPredictions_mobileandfbb" : "6.2",
                  "getModelPredictions_others" : "6.3", "dp_levers_onlymob" : "7.1","df_levers_onlymob" : "7.1", "dp_levers_mobileandfbb" : "7.2", "df_levers_mobileandfbb" : "7.2", "dp_levers_others" : "7.3", "df_levers_others" : "7.3",
                  "create_delivery_table" : "8", "create_delivery_file" : "9", "run_delivery": "10", "build_delivery_df":"7"}

df = spark.read.table("tests_es.churn_log_times").where(col("closing_day").isin([closing_day, prev_closing_day, prev_prev_closing_day]))
df = df.where(col("duration")!=-1).sort(asc("end_time"))
df = df.withColumn("step_father", col("step_name")).replace(to_replace=step_names_dict, subset=['step_father']) 
df = df.withColumn("step_number", col("step_father")).replace(to_replace=step_order_dict, subset=['step_number']).withColumn("step_number", col("step_number").cast("integer")) 
df = df.withColumn("duration_mins", col("duration")/60)



In [141]:
df.groupBy("step_number", 'step_father', 'step_name').pivot("process_name").agg(
                            sql_sum("duration_mins").cast("int").alias("duration_mins")).sort(asc("step_number")).show(100, truncate=False)



+-----------+--------------------------------+--------------------------------+-----------------------+-----------------------+-----------------------+
|step_number|step_father                     |step_name                       |churn_delivery_20190514|churn_delivery_20190521|churn_delivery_20190531|
+-----------+--------------------------------+--------------------------------+-----------------------+-----------------------+-----------------------+
|1          |car                             |usage_geneva_voice              |90                     |67                     |109                    |
|1          |car                             |usage_geneva_data               |99                     |76                     |153                    |
|1          |car                             |device_catalogue                |3                      |7                      |3                      |
|1          |car                             |call_centre_calls               |14       

In [ ]:
df.where(col("closing_day")==closing_day).select("step_name", "duration_mins").sort(desc("duration_mins")).show(100,False)

In [ ]:
times = {}

for cc in [closing_day, prev_closing_day, prev_prev_closing_day]
    times[cc] = df.where(col("closing_day")==cc).select(sql_sum("duration")/3600).rdd.collect()[0]
times

In [ ]:
df_delivery = spark.read.table("tests_es.churn_team_delivery_20190514_prepared").drop("msisdn").withColumnRenamed("msisdn_a", "msisdn")
df_delivery.select("Incertidumbre").distinct().show()

In [ ]:

from pyspark.sql.functions import concat_ws, length, year, month, dayofmonth, regexp_replace, split
import datetime as dt
from churn.datapreparation.general.model_outputs_manager import ensure_types_model_scores_columns
from pyspark.sql.types import IntegerType

closing_day = "20190507"

from churn.delivery.delivery_constants import *
from churn.delivery.model_outputs_formatter import create_model_output_dataframes
df_param, df_scores = create_model_output_dataframes(spark, closing_day)

# from churn.datapreparation.general.model_outputs_manager import insert_to_model_scores, insert_to_model_parameters
# insert_to_model_scores(df_scores)
# insert_to_model_parameters(df_param)

